<a href="https://colab.research.google.com/github/nogutikeisuke/DeepLearninng2022Late/blob/main/%E5%8F%8E%E9%9B%86%E3%81%97%E3%81%9F%E7%94%BB%E5%83%8F%E3%81%A7%E5%AD%A6%E7%BF%92%E3%81%A8%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
os.makedirs('sample',exist_ok=True)
os.makedirs('output/dog/sample',exist_ok=True)
os.makedirs('output/cat/sample',exist_ok=True)
os.makedirs('img/deep_learning/dog',exist_ok=True)
os.makedirs('img/deep_learning/cat',exist_ok=True)

In [9]:
import cv2
import numpy as np
import glob as glob
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

In [10]:
path="img/deep_learning"
folders=os.listdir(path)
classes = [f for f in folders if os.path.isdir(os.path.join(path,f))]
print(classes)
n_classes = len (classes)
print(n_classes)

['dog', 'cat']
2


In [11]:
X=[]
Y=[]
for label,class_name in enumerate(classes):
  files = glob.glob(path+"/"+class_name+"/*jpg")
  for file in files:
    img =cv2.imread(file)
    img =cv2.resize(img,dsize=(224,224))
    X.append(img)
    Y.append(label)

In [12]:
X=np.array(X)
X=X.astype('float32')
X/=255.0
Y=np.array(Y)
Y=np_utils.to_categorical(Y,n_classes)

In [13]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(120, 224, 224, 3)
(31, 224, 224, 3)
(120, 2)
(31, 2)


In [14]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input,Activation,Dense,Flatten,Dropout
from keras.optimizers import Adam

In [15]:
input_tensor=Input(shape=(224,224,3))
base_model=VGG16(weights='imagenet',input_tensor=input_tensor,include_top=False)

58889256/58889256 [==============================] - 2s 0us/step


In [16]:
top_model=Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(n_classes,activation='softmax'))

In [17]:
model =Model(inputs=base_model.input,outputs=top_model(base_model.output))

In [18]:
for layer in model.layers[:15]:
  layer.trainable=False
  print('# layer=',len(model.layers))

# layer= 20
# layer= 20
# layer= 20
# layer= 20
# layer= 20
# layer= 20
# layer= 20
# layer= 20
# layer= 20
# layer= 20
# layer= 20
# layer= 20
# layer= 20
# layer= 20
# layer= 20


In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',metrics=['accuracy'])

In [20]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [21]:
model.fit(X_train,Y_train,epochs=20,batch_size=16)

Epoch 1/20
8/8 [==============================] - 11s 233ms/step - loss: 2.8920 - accuracy: 0.4917
Epoch 2/20
8/8 [==============================] - 1s 86ms/step - loss: 1.5292 - accuracy: 0.5000
Epoch 3/20
8/8 [==============================] - 1s 87ms/step - loss: 0.7276 - accuracy: 0.5333
Epoch 4/20
8/8 [==============================] - 1s 88ms/step - loss: 0.7018 - accuracy: 0.5333
Epoch 5/20
8/8 [==============================] - 1s 85ms/step - loss: 0.6931 - accuracy: 0.5333
Epoch 6/20
8/8 [==============================] - 1s 87ms/step - loss: 0.6928 - accuracy: 0.5333
Epoch 7/20
8/8 [==============================] - 1s 86ms/step - loss: 0.6927 - accuracy: 0.5333
Epoch 8/20
8/8 [==============================] - 1s 87ms/step - loss: 0.6928 - accuracy: 0.5333
Epoch 9/20
8/8 [==============================] - 1s 87ms/step - loss: 0.6926 - accuracy: 0.5333
Epoch 10/20
8/8 [==============================] - 1s 87ms/step - loss: 0.6925 - accuracy: 0.5333
Epoch 11/20
8/8 [==========

In [22]:
score=model.evaluate(X_test,Y_test,batch_size=16)

2/2 [==============================] - 2s 2s/step - loss: 0.6997 - accuracy: 0.3871


In [23]:
import pickle
pickle.dump(classes,open('classes.sav','wb'))
model.save('cnn.h5')

In [24]:
from keras.models import load_model
import pickle
import cv2
import glob

In [25]:
model=load_model('cnn.h5')
classes=pickle.load(open('classes.sav','rb'))

In [29]:
files=glob.glob('sample/*')
print(files)

['sample/dog_5.jpg', 'sample/dog_6.jpg', 'sample/dog_65.jpg']


In [37]:
img=cv2.imread(file)
img2=cv2.resize(img,dsize=(224,224))
img2=img2.astype('float32')
img2=img2/255.0
img2=img2[None,...]
result=model.predict(img2)
print(result)
pred=result.argmax()
print(pred,str(classes[pred]))
cv2.imwrite('output/'+str(classes[pred])+'/'+file,img)

1/1 [==============================] - 0s 17ms/step
[[0.5136269  0.48637313]]
0 dog


False